In [23]:
def sudoku(f):

    def af(g):
        for n, l in enumerate(g):
            for m, c in enumerate(l):
                P(str(c).replace("0", "."), end="")
                if m in {2, 5}:
                    P("+", end="")
            P()
            if n in {2, 5}:
                P("+" * 11)

    def cp(q, s):
        l = set(s[q[0]])
        l |= {s[i][q[1]] for i in range(9)}
        k = q[0] // 3, q[1] // 3
        for i in range(3):
            l |= set(s[k[0] * 3 + i][k[1] * 3:(k[1] + 1) * 3])
        return set(range(1, 10)) - l

    def ec(l):
        q = set(l) - {0}
        for c in q:
            if l.count(c) != 1:
                return True
        return False

    P = print
    af(f)

    s = []
    t = []
    for nl, l in enumerate(f):
        try:
            n = list(map(int, l))
        except:
            P("La ligne " + str(nl + 1) + " contient autre chose qu'un chiffre.")
            return
        if len(n) != 9:
            P("La ligne " + str(nl + 1) + " ne contient pas 9 chiffres.")
            return
        t += [[nl, i] for i in range(9) if n[i] == 0]
        s.append(n)
    if nl != 8:
        P("Le jeu contient " + str(nl + 1) + " lignes au lieu de 9.")
        return

    for l in range(9):
        if ec(s[l]):
            P("La ligne " + str(l + 1) + " est contradictoire.")
            return
    for c in range(9):
        k = [s[l][c] for l in range(9)]
        if ec(k):
            P("La colonne " + str(c + 1) + " est contradictoire.")
            return
    for l in range(3):
        for c in range(3):
            q = []
            for i in range(3):
                q += s[l * 3 + i][c * 3:(c + 1) * 3]
            if ec(q):
                P("La cellule (" + str(l + 1) + ";" +
                  str(c + 1) + ") est contradictoire.")
                return

    p = [[] for i in t]
    cr = 0

    while cr < len(t):
        p[cr] = cp(t[cr], s)
        try:
            while not p[cr]:
                s[t[cr][0]][t[cr][1]] = 0
                cr -= 1
        except:
            P("Le sudoku n'a pas de solution.")
            return
        s[t[cr][0]][t[cr][1]] = p[cr].pop()
        cr += 1

    af(s)
    return(s)

In [26]:


import cv2
import numpy as np
import operator
from keras.models import load_model
from keras.models import model_from_json


classifier = load_model("C:/digit_model.h5")

marge = 4
case = 28 + 2 * marge
taille_grille = 9 * case

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
flag = 0
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (1080, 620))


while True:

    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 2)

    contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_grille = None
    maxArea = 0

    for c in contours:
        area = cv2.contourArea(c)
        if area > 25000:
            peri = cv2.arcLength(c, True)
            polygone = cv2.approxPolyDP(c, 0.01 * peri, True)
            if area > maxArea and len(polygone) == 4:
                contour_grille = polygone
                maxArea = area

    if contour_grille is not None:
        cv2.drawContours(frame, [contour_grille], 0, (0, 255, 0), 2)
        points = np.vstack(contour_grille).squeeze()
        points = sorted(points, key=operator.itemgetter(1))
        if points[0][0] < points[1][0]:
            if points[3][0] < points[2][0]:
                pts1 = np.float32([points[0], points[1], points[3], points[2]])
            else:
                pts1 = np.float32([points[0], points[1], points[2], points[3]])
        else:
            if points[3][0] < points[2][0]:
                pts1 = np.float32([points[1], points[0], points[3], points[2]])
            else:
                pts1 = np.float32([points[1], points[0], points[2], points[3]])
        pts2 = np.float32([[0, 0], [taille_grille, 0], [0, taille_grille], [
                          taille_grille, taille_grille]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        grille = cv2.warpPerspective(frame, M, (taille_grille, taille_grille))
        grille = cv2.cvtColor(grille, cv2.COLOR_BGR2GRAY)
        grille = cv2.adaptiveThreshold(
            grille, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 7, 3)

        cv2.imshow("grille", grille)
        if flag == 0:

            grille_txt = []
            for y in range(9):
                ligne = ""
                for x in range(9):
                    y2min = y * case + marge
                    y2max = (y + 1) * case - marge
                    x2min = x * case + marge
                    x2max = (x + 1) * case - marge
                    cv2.imwrite("mat" + str(y) + str(x) + ".png",
                                grille[y2min:y2max, x2min:x2max])
                    img = grille[y2min:y2max, x2min:x2max]
                    x = img.reshape(1, 28, 28, 1)
                    if x.sum() > 10000:
                        prediction = classifier.predict_classes(x)
                        ligne += "{:d}".format(prediction[0])
                    else:
                        ligne += "{:d}".format(0)
                grille_txt.append(ligne)
            print(grille_txt)
            result = sudoku(grille_txt)
        print("Resultat:", result)

        if result is not None:
            flag = 1
            fond = np.zeros(
                shape=(taille_grille, taille_grille, 3), dtype=np.float32)
            for y in range(len(result)):
                for x in range(len(result[y])):
                    if grille_txt[y][x] == "0":
                        cv2.putText(fond, "{:d}".format(result[y][x]), ((
                            x) * case + marge + 3, (y + 1) * case - marge - 3), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 0.9, (0, 0, 255), 1)
            M = cv2.getPerspectiveTransform(pts2, pts1)
            h, w, c = frame.shape
            fondP = cv2.warpPerspective(fond, M, (w, h))
            img2gray = cv2.cvtColor(fondP, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask = mask.astype('uint8')
            mask_inv = cv2.bitwise_not(mask)
            img1_bg = cv2.bitwise_and(frame, frame, mask=mask_inv)
            img2_fg = cv2.bitwise_and(fondP, fondP, mask=mask).astype('uint8')
            dst = cv2.add(img1_bg, img2_fg)
            dst = cv2.resize(dst, (1080, 620))
            cv2.imshow("frame", dst)
            out.write(dst)

        else:
            frame = cv2.resize(frame, (1080, 620))
            cv2.imshow("frame", frame)
            out.write(frame)

    else:
        flag = 0
        frame = cv2.resize(frame, (1080, 620))
        cv2.imshow("frame", frame)
        out.write(frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break


out.release()
cap.release()
cv2.destroyAllWindows()

['000000000', '500000000', '400600000', '900603000', '900601000', '940603000', '905601000', '922252220', '000000000']
...+...+...
5..+...+...
4..+6..+...
+++++++++++
9..+6.3+...
9..+6.1+...
94.+6.3+...
+++++++++++
9.5+6.1+...
922+252+22.
...+...+...
La ligne 8 est contradictoire.
Resultat: None
['070020036', '050000990', '200200785', '085055000', '720000059', '000530580', '557002008', '058000060', '630080070']
.7.+.2.+.36
.5.+...+99.
2..+2..+785
+++++++++++
.85+.55+...
72.+...+.59
...+53.+58.
+++++++++++
557+..2+..8
.58+...+.6.
63.+.8.+.7.
La ligne 2 est contradictoire.
Resultat: None
['070020086', '060000890', '200600715', '082097000', '710000059', '000130480', '697002008', '058000060', '430080070']
.7.+.2.+.86
.6.+...+89.
2..+6..+715
+++++++++++
.82+.97+...
71.+...+.59
...+13.+48.
+++++++++++
697+..2+..8
.58+...+.6.
43.+.8.+.7.
La colonne 8 est contradictoire.
Resultat: None
['070020046', '060000890', '200900715', '084097000', '710000059', '000130480', '697002008', '058000060', '4300

['070020086', '060000890', '200800785', '082097000', '710000059', '000130880', '697002008', '058000060', '430080070']
.7.+.2.+.86
.6.+...+89.
2..+8..+785
+++++++++++
.82+.97+...
71.+...+.59
...+13.+88.
+++++++++++
697+..2+..8
.58+...+.6.
43.+.8.+.7.
La ligne 3 est contradictoire.
Resultat: None
['070020036', '060000380', '200600735', '086097000', '710000053', '000130880', '697002008', '058000060', '430080070']
.7.+.2.+.36
.6.+...+38.
2..+6..+735
+++++++++++
.86+.97+...
71.+...+.53
...+13.+88.
+++++++++++
697+..2+..8
.58+...+.6.
43.+.8.+.7.
La ligne 6 est contradictoire.
Resultat: None
['070020046', '060000880', '200800715', '086097000', '710000059', '000130880', '697002008', '058000060', '430080070']
.7.+.2.+.46
.6.+...+88.
2..+8..+715
+++++++++++
.86+.97+...
71.+...+.59
...+13.+88.
+++++++++++
697+..2+..8
.58+...+.6.
43.+.8.+.7.
La ligne 2 est contradictoire.
Resultat: None
['070020036', '060000880', '200800735', '086097000', '710000059', '000130480', '697002008', '058000060', '430080

['070020046', '060000890', '200800715', '086097000', '770000059', '000130480', '697002008', '058000060', '830080070']
.7.+.2.+.46
.6.+...+89.
2..+8..+715
+++++++++++
.86+.97+...
77.+...+.59
...+13.+48.
+++++++++++
697+..2+..8
.58+...+.6.
83.+.8.+.7.
La ligne 5 est contradictoire.
Resultat: None
['050820086', '050000890', '200300715', '686097000', '710000059', '000130380', '697002008', '058000060', '430080070']
.5.+82.+.86
.5.+...+89.
2..+3..+715
+++++++++++
686+.97+...
71.+...+.59
...+13.+38.
+++++++++++
697+..2+..8
.58+...+.6.
43.+.8.+.7.
La ligne 1 est contradictoire.
Resultat: None
['000000000', '177755771', '140609901', '844545661', '130609081', '142259541', '155609071', '122293221', '000000000']
...+...+...
177+755+771
14.+6.9+9.1
+++++++++++
844+545+661
13.+6.9+.81
142+259+541
+++++++++++
155+6.9+.71
122+293+221
...+...+...
La ligne 2 est contradictoire.
Resultat: None
['070020046', '060000890', '200800795', '086097000', '770000059', '000130480', '697002008', '058000060', '830080

['026030008', '900600800', '000039040', '007302000', '004070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+8..
...+.39+.4.
+++++++++++
..7+3.2+...
..4+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La colonne 5 est contradictoire.
Resultat: None
['026030008', '900600800', '000019080', '007302000', '006070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+8..
...+.19+.8.
+++++++++++
..7+3.2+...
..6+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La colonne 3 est contradictoire.
Resultat: None
['026030008', '900600800', '000099040', '007302000', '006070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+8..
...+.99+.4.
+++++++++++
..7+3.2+...
..6+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La ligne 3 est contradictoire.
Resultat: None
['026030008', '900600500', '000019040', '007302000', '004070800', '000806700', '050720000', '009005004', '40

['026030008', '900600900', '000089030', '007302000', '008070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+9..
...+.89+.3.
+++++++++++
..7+3.2+...
..8+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La ligne 2 est contradictoire.
Resultat: None
['026030008', '900600800', '000088080', '007302000', '004070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+8..
...+.88+.8.
+++++++++++
..7+3.2+...
..4+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La ligne 3 est contradictoire.
Resultat: None
['026030008', '900800800', '000038040', '007302000', '004070800', '000808700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+8..+8..
...+.38+.4.
+++++++++++
..7+3.2+...
..4+.7.+8..
...+8.8+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La ligne 2 est contradictoire.
Resultat: None
['026030008', '900600300', '000039030', '007302000', '002070800', '000806700', '050720000', '009005004', '400060

...

Resultat: [[1, 5, 6, 2, 3, 9, 7, 8, 4], [9, 3, 4, 7, 6, 8, 1, 5, 2], [7, 8, 2, 1, 4, 5, 9, 6, 3], [3, 7, 5, 4, 1, 2, 8, 9, 6], [6, 4, 1, 8, 9, 7, 2, 3, 5], [2, 9, 8, 3, 5, 6, 4, 1, 7], [4, 6, 7, 9, 8, 3, 5, 2, 1], [5, 1, 9, 6, 2, 4, 3, 7, 8], [8, 2, 3, 5, 7, 1, 6, 4, 9]]
Resultat: [[1, 5, 6, 2, 3, 9, 7, 8, 4], [9, 3, 4, 7, 6, 8, 1, 5, 2], [7, 8, 2, 1, 4, 5, 9, 6, 3], [3, 7, 5, 4, 1, 2, 8, 9, 6], [6, 4, 1, 8, 9, 7, 2, 3, 5], [2, 9, 8, 3, 5, 6, 4, 1, 7], [4, 6, 7, 9, 8, 3, 5, 2, 1], [5, 1, 9, 6, 2, 4, 3, 7, 8], [8, 2, 3, 5, 7, 1, 6, 4, 9]]
Resultat: [[1, 5, 6, 2, 3, 9, 7, 8, 4], [9, 3, 4, 7, 6, 8, 1, 5, 2], [7, 8, 2, 1, 4, 5, 9, 6, 3], [3, 7, 5, 4, 1, 2, 8, 9, 6], [6, 4, 1, 8, 9, 7, 2, 3, 5], [2, 9, 8, 3, 5, 6, 4, 1, 7], [4, 6, 7, 9, 8, 3, 5, 2, 1], [5, 1, 9, 6, 2, 4, 3, 7, 8], [8, 2, 3, 5, 7, 1, 6, 4, 9]]
Resultat: [[1, 5, 6, 2, 3, 9, 7, 8, 4], [9, 3, 4, 7, 6, 8, 1, 5, 2], [7, 8, 2, 1, 4, 5, 9, 6, 3], [3, 7, 5, 4, 1, 2, 8, 9, 6], [6, 4, 1, 8, 9, 7, 2, 3, 5], [2, 9, 8, 3, 5, 6, 4, 1, 7],

Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7],

Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7],

Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7], [6, 9, 7, 4, 1, 2, 5, 3, 8], [1, 5, 8, 3, 7, 9, 2, 6, 4], [4, 3, 2, 5, 8, 6, 9, 7, 1]]
Resultat: [[8, 7, 5, 9, 2, 1, 3, 4, 6], [3, 6, 1, 7, 5, 4, 8, 9, 2], [2, 4, 9, 8, 6, 3, 7, 1, 5], [5, 8, 4, 6, 9, 7, 1, 2, 3], [7, 1, 3, 2, 4, 8, 6, 5, 9], [9, 2, 6, 1, 3, 5, 4, 8, 7],

['026030008', '900600100', '000039040', '007302000', '002070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+1..
...+.39+.4.
+++++++++++
..7+3.2+...
..2+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La colonne 5 est contradictoire.
Resultat: None
['026030008', '900600100', '000019080', '007302000', '005070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+1..
...+.19+.8.
+++++++++++
..7+3.2+...
..5+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La cellule (1;3) est contradictoire.
Resultat: None
['026030008', '900600100', '000039030', '007302000', '004070800', '000806700', '050720000', '009005004', '400060210']
.26+.3.+..8
9..+6..+1..
...+.39+.3.
+++++++++++
..7+3.2+...
..4+.7.+8..
...+8.6+7..
+++++++++++
.5.+72.+...
..9+..5+..4
4..+.6.+21.
La ligne 3 est contradictoire.
Resultat: None
['026030008', '900600100', '000019040', '007302000', '006070800', '000806700', '050720000', '009005004',